**INSERT…ON CONFLICT (Upsert) :**

Starting with version 9.5, PostgreSQL allows *“upserts”* (update or insert) of rows into a table via the *ON CONFLICT clause of the INSERT* statement.A candidate row will only be inserted if that row does not violate any unique constraints.

In the case of a unique constraint violation, a secondary action can occur:
 - “DO UPDATE”
 - “DO NOTHING” : silently skip this row.

Conflicts are determined using existing *unique constraints and indexes*. 

SQLAlchemy provides ON CONFLICT support via the *PostgreSQL-specific insert()* function, which provides the generative methods Insert.on_conflict_do_update() and Insert.on_conflict_do_nothing():

In [2]:
from sqlalchemy.dialects.postgresql import insert
from sqlalchemy import Table, Column, MetaData, Integer, String, create_engine, URL


url = URL.create(
    drivername="postgresql",
    username="postgres",
    password="1234",
    host="192.168.0.8",
    port="5432",
    database="tests"
)

engine = create_engine(url)

metadata = MetaData()
table = Table(
    "data",
    metadata,
    Column('id', Integer, primary_key=True),
    Column('data', String)
)


insert_stmt = insert(table).values(
    id= 1, # some_existing_id
    data='some new data') # inserted value
do_nothing_stmt = insert_stmt.on_conflict_do_nothing(
    index_elements=['id']
)
print(do_nothing_stmt)

do_update_stmt = insert_stmt.on_conflict_do_update(
    constraint='data_pkey',
    set_=dict(data='updated data')
)
print(do_update_stmt)

with engine.begin() as conn: 
    conn.execute(do_nothing_stmt)
    result =  conn.execute(table.select().where(table.c.data == "some new data"))
    print(result.fetchall())

    conn.execute(do_update_stmt)
    

INSERT INTO data (id, data) VALUES (%(id)s, %(data)s) ON CONFLICT (id) DO NOTHING
INSERT INTO data (id, data) VALUES (%(id)s, %(data)s) ON CONFLICT ON CONSTRAINT data_pkey DO UPDATE SET data = %(param_1)s
[]
